In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 12
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.3e6
length = 50

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e

    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 22585916.0, E: 84.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2300000.00, score:-171.96
 Observe Time: 0 | S: 22585916.0, E: 84.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:39.7, cum_e:39.7, new_i:35.5, cum_i:35.5
 True State Time: 1 | S: 22585849.9, E: 110.9, A: 0.0, I: 35.5, D: 0.0, R: 3.8, Budget: 2300000.00, score:-415.76
 Observe Time: 1 | S: 22585851.0, E: 108.0, A: 0.0, I: 35.5, D: 0.0, R: 0.0
 Time: 1 | new_e:55.7, cum_e:95.4, new_i:49.0, cum_i:84.5
 True State Time: 2 | S: 22585757.0, E: 149.7, A: 0.0, I: 84.4, D: 0.0, R: 8.8, Budget: 2300000.00, score:-762.30
 Observe Time: 2 | S: 22585761.0, E: 151.0, A: 0.0, I: 84.4, D: 0.0, R: 0.0
 Time: 2 | new_e:78.7, cum_e:174.1, new_i:68.5, cum_i:153.0
 True State Time: 3 | S: 22585625.9, E: 205.5, A: 0.0, I: 152.8, D: 0.1, R: 15.6, Budget: 2300000.00, score:-1255.66
 Observe Time: 3 | S: 22585632.0, E: 206.0, A: 0.0, I: 152.8, D: 0.1, R: 9.0
 Time: 3 | new_e:111.5, cum_e:285.6, new_i:96.4, cum_i:249.3
 Tr

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 23 | S: 21568664.5, E: 7.6, A: 188.6, I: 577.9, D: 3.8, R: 1016557.6, Budget: 1533171.19, score:-8266.89
 Observe Time: 23 | S: 21568771.0, E: 11.0, A: 188.6, I: 577.9, D: 3.8, R: 1016554.0
 Time: 23 | new_e:3.4, cum_e:1854.6, new_i:23.3, cum_i:2478.4
 True State Time: 24 | S: 21510064.7, E: 3.5, A: 166.2, I: 529.0, D: 3.8, R: 1075232.9, Budget: 1509510.31, score:-8277.59
 Observe Time: 24 | S: 21510169.0, E: 0.0, A: 166.2, I: 529.0, D: 3.8, R: 1075228.0
 Time: 24 | new_e:1.6, cum_e:1856.2, new_i:19.2, cum_i:2497.6
 True State Time: 25 | S: 21469887.9, E: 3.0, A: 142.4, I: 482.0, D: 3.9, R: 1115

 True State Time: 42 | S: 20764590.3, E: 4.1, A: 24.7, I: 110.6, D: 4.6, R: 1821265.7, Budget: 992503.10, score:-8611.54
 Observe Time: 42 | S: 20764743.0, E: 0.0, A: 24.7, I: 110.6, D: 4.6, R: 1821263.0
 Time: 42 | new_e:3.8, cum_e:1923.7, new_i:5.6, cum_i:2672.3
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 43 | S: 20720500.3, E: 6.0, A: 22.1, I: 103.8, D: 4.6, R: 1865363.0, Budget: 962321.92, score:-8623.56
 Observe Time: 43 | S: 20720655.0, E: 1.0, A: 22.1, I: 103.8, D: 4.6, R: 1865361.0
 Time: 43 | new_e:2.6, cum_e:1926.3, new_i:4.8, cum_i:2677.1
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 44 | S: 20675731.9, E: 4.6, A: 21.9, I: 95.6, D: 4.6, R: 1910141.4, Budg

 True State Time: 65 | S: 20005730.5, E: 13.1, A: 11.6, I: 42.4, D: 4.8, R: 2580197.6, Budget: 501544.44, score:-8950.00
 Observe Time: 65 | S: 20005985.0, E: 0.0, A: 11.6, I: 42.4, D: 4.8, R: 2580201.0
 Time: 65 | new_e:5.4, cum_e:1999.0, new_i:6.3, cum_i:2776.8
 True State Time: 66 | S: 19984510.6, E: 14.4, A: 11.9, I: 43.3, D: 4.8, R: 2601415.0, Budget: 483677.38, score:-8976.08
 Observe Time: 66 | S: 19984774.0, E: 0.0, A: 11.9, I: 43.3, D: 4.8, R: 2601418.0
 Time: 66 | new_e:5.9, cum_e:2005.0, new_i:6.8, cum_i:2783.6
 True State Time: 67 | S: 19956566.6, E: 15.3, A: 12.9, I: 44.7, D: 4.8, R: 2629355.7, Budget: 468206.54, score:-9003.80
 Observe Time: 67 | S: 19956840.0, E: 0.0, A: 12.9, I: 44.7, D: 4.8, R: 2629360.0
 Time: 67 | new_e:6.3, cum_e:2011.3, new_i:7.2, cum_i:2790.8
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 

 True State Time: 86 | S: 19585481.2, E: 308.3, A: 72.1, I: 424.7, D: 5.2, R: 2999708.6, Budget: 226856.36, score:-12528.46
 Observe Time: 86 | S: 19586860.0, E: 0.0, A: 72.1, I: 424.7, D: 5.2, R: 2999654.0
 Time: 86 | new_e:124.3, cum_e:2818.6, new_i:123.6, cum_i:3603.5
 True State Time: 87 | S: 19581750.6, E: 370.2, A: 76.4, I: 495.1, D: 5.2, R: 3003302.3, Budget: 217773.54, score:-13178.50
 Observe Time: 87 | S: 19583334.0, E: 0.0, A: 76.4, I: 495.1, D: 5.2, R: 3003235.0
 Time: 87 | new_e:149.2, cum_e:2967.8, new_i:147.3, cum_i:3750.8
 True State Time: 88 | S: 19572613.9, E: 442.4, A: 85.3, I: 580.5, D: 5.3, R: 3012272.6, Budget: 209318.90, score:-13954.87
 Observe Time: 88 | S: 19574444.0, E: 0.0, A: 85.3, I: 580.5, D: 5.3, R: 3012193.0
 Time: 88 | new_e:178.2, cum_e:3146.0, new_i:175.3, cum_i:3926.2
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeas

In [7]:
(time.time() - tic) / ( T - start_time + 1)

84.50402925715727